### Task 1
Construct the value-weighted market return using CRSP data, replicating the market return time series available in Kenneth French website. Also calculate the equal-weighted market return, and the lagged total market capitalization. Your output should be from January 1926 to Decembe 2017, at a monthly frequency.


### Approach –

### Data Source 
WRDS CRSP > Annual Update > Stock / Security Files > CRSP Monthly Stock.
File imported in R and date is formatted as YYYY-MM-DD (R date format). And passed it as input
to PS_Q1 function.

### Data cleaning steps:

-  1. Universe of stocks: Following Ken French procedure, I restrict the sample to common shares
(share codes 10 and 11) and to securities traded in the New York Stock Exchange, American
Stock Exchange, or the Nasdaq Stock Exchange (exchange codes 1, 2, and 3).<br>

-  2. Missing returns: Returns which are marked with characters(non-numeric) due to reasons viz.
No valid comparison for an excess return, no listing information, no valid previous price, Offexchange,
Out of Range, no valid price has been marked as 0. They have not been deleted to
avoid discontinuity in data as some of the calculation requires one month lagged calculation. <br>

-  3. Delisting return calculation: All non-numeric delisting returns have been converted to 0 <br>

-  4. Adjusted holding period Return: Following method has been used to calculated adjusted
holding period return from “holding period return” rh_i,t and delisting return rd_i,t.<br>

-  5. Negative Price: Negative price in CRSP data is bid-ask average. Absolute value of price has
been taken.<br>

-  6. NA or missing value in price, share outstanding: All these NA has been converted to 0.<br>

-  7. Market Capitalization: price of stock has been multiplied with outstanding shares<br>

-  8. Portfolio weights: Weight for each stock has been calculated dividing market cap of each stock
by sum of total market cap for each month.<br>

-  9. Sample period: Sample period taken is Jan 1926 to Dec 2017.<br>

-  10. Calculation of Value Weighted Return:
Value Weighted Return = (Market Cap of stock last month/Sum of Market Cap of all the stock last month) ∗ 𝑟𝑒𝑡𝑢𝑟𝑛 𝑜𝑓 𝑠𝑡𝑜𝑐𝑘 𝑡ℎ𝑖𝑠 𝑚𝑜𝑛𝑡ℎ<br>

-  11. Calculation of equal weighted return:
Equal Weighted Return = (1/Number of Stocks in the market)∗ 𝑟𝑒𝑡𝑢𝑟𝑛 𝑜𝑓 𝑠𝑡𝑜𝑐𝑘 𝑡ℎ𝑖𝑠 𝑚𝑜𝑛𝑡ℎ<br>

In [ ]:
if (!require(data.table)) install.packages("data.table")
if (!require(tidyr)) install.packages("tidyr")
if (!require(dplyr)) install.packages("dplyr")
if (!require(moments)) install.packages("moments")


CRSP_Stocks <- fread("./CRSP_Stocks.csv", header = TRUE)
CRSP_Stocks$date<- as.Date(as.character(CRSP_Stocks$date), "%Y%m%d")

PS1_Q1 <- function(CRSP_Stocks) {
  
  ### datacleaning
  
  #Taking only Common shares (share code 10, 11) and Stocks traded in NYSE, AMEX, NASDAQ
  CRSP_Stocks <- CRSP_Stocks[(CRSP_Stocks$SHRCD %in% c(10, 11))  & (CRSP_Stocks$EXCHCD %in% c(1, 2, 3))]
  
  ## Number of share outstanding is in 1000 numbers so multiplying by 1000
  CRSP_Stocks[, SHROUT:=SHROUT*1000, ]
  
  
  #holding period return
  #C stands for No valid previous price. Setting return to 0
  CRSP_Stocks$RET[CRSP_Stocks$RET=="C"] = 0 
  #Convert non numeric holding period returns to NA
  CRSP_Stocks$RET <- as.numeric(as.character(CRSP_Stocks$RET))
  CRSP_Stocks$RET[is.na(CRSP_Stocks$RET)] <- 0
  
  #Negative price are bid-ask average
  CRSP_Stocks$PRC = abs(CRSP_Stocks$PRC)
  CRSP_Stocks$PRC <- as.numeric(as.character(CRSP_Stocks$PRC))
  CRSP_Stocks$PRC[is.na(CRSP_Stocks$PRC)] <- 0
  
  #Delist Return
  #Change all special cases to NA
  CRSP_Stocks$DLRET <- as.numeric(as.character(CRSP_Stocks$DLRET))
  CRSP_Stocks$adjRET = ifelse(!is.na(CRSP_Stocks$RET), ifelse(is.na(CRSP_Stocks$DLRET), CRSP_Stocks$RET,((1+CRSP_Stocks$DLRET)*(1+CRSP_Stocks$RET)-1) ), CRSP_Stocks$DLRET)
  CRSP_Stocks$adjRET[is.na(CRSP_Stocks$adjRET)] <- 0
  
  #Shareoutstanding make NA as 0
  CRSP_Stocks$SHROUT[is.na(CRSP_Stocks$SHROUT)] <- 0
  
  ### Calculation 
  CRSP_Stocks[, yearmonth:=format(as.Date(date), "%Y-%m") ]
  CRSP_Stocks$market_cap <- CRSP_Stocks$PRC*CRSP_Stocks$SHROUT
  
  CRSP_Stocks[, sum_market_cap:=sum(market_cap), by=yearmonth]
  CRSP_Stocks[, vw_weight:=market_cap/sum_market_cap]
  
  setorder(CRSP_Stocks, PERMNO, yearmonth)
  #CRSP_Stocks[, Stock_lag_MV:= shift(sum_market_cap), by=yearmonth]
  CRSP_Stocks[, vw_weight_lastMonth:= shift(vw_weight), by=PERMNO]
  
  CRSP_Stocks[, vwret:=vw_weight_lastMonth*adjRET]
  CRSP_Stocks[, Stock_Vw_Ret:=sum(vwret, na.rm = TRUE), by=yearmonth]
  CRSP_Stocks[, Stock_Ew_Ret:=mean(adjRET), by=yearmonth]
  
  PS1_Q1_output <- CRSP_Stocks[, c("yearmonth", "Stock_Ew_Ret", "Stock_Vw_Ret", "sum_market_cap")]
  PS1_Q1_output=PS1_Q1_output[!duplicated(yearmonth)]
  
  setorder(PS1_Q1_output, yearmonth)
  PS1_Q1_output[, Stock_lag_MV:= shift(sum_market_cap)]
  
  PS1_Q1_output <- separate(PS1_Q1_output, yearmonth, into = c("Year", "Month"), sep="-")
  PS1_Q1_output$Stock_lag_MV <- PS1_Q1_output$Stock_lag_MV/1000000
  PS1_Q1_output <- PS1_Q1_output[complete.cases(PS1_Q1_output),]
  setorder(PS1_Q1_output, Year, Month)
  PS1_Q1_output <- PS1_Q1_output[, c("Year", "Month", "Stock_lag_MV", "Stock_Ew_Ret", "Stock_Vw_Ret")]
  
  
  
  return(PS1_Q1_output)
  
}

Monthly_CRSP_Stocks <- PS1_Q1(CRSP_Stocks)


## Task 2
Using the risk-free rate of return from French’s website, report the following moments of the market
excess returns for both time series (4 decimal digits): annualized return, annualized volatility,
annualized Sharpe ratio, skewness, and excess kurtosis. Annualized values should be calculated
geometrically. You should be comparing between July 1926 to December 2017, at a monthly frequency.

For this analysis, we downloaded the Fama/French 3 Factors file (FF_
Research_Data_Factors.csv).
Following five statistics have been calculated for CRSP stock data and data from FF-3 Factor file.
Column 1 shows Annualized Mean, Annualized Standard Deviation, Annualized Sharp Ratio,
Skewness, Excess Kurtosis for CRSP Monthly Stock data. While the column 2 shows those
statistics for FF-3 Factor file data.

-  1. Sample period: Monthly from July 1926 to December 2017. Since the FF-3 Factor file starts
from July 1926 so CRSP data has been truncated to have the same period.

-  2. Skewness: Skewness has been calculated using skewness function in R moment package on
full monthly time series data. There is no annualization or log scaling in calculation of Excess
Skewness.

-  3. Excess Kurtosis: Using Kurtosis function in moment package, Kurtosis has been calculated
for entire monthly data (without any annualization or log scaling). To calculate excess kurtosis,
I have subtracted 3 from the Kurtosis value.

In [ ]:
FF_mkt <- fread("./F-F_Research_Data_Factors.CSV", skip=3, header = TRUE)
FF_mkt <- FF_mkt[, .(date=V1, `Mkt-RF` = `Mkt-RF`/100, SMB = SMB/100, HML=HML/100, RF=RF/100)]



PS1_Q2 <- function(CRSP_Stocks, FF_mkt){
  
  PS1_Q1_output <- PS1_Q1(CRSP_Stocks)
  PS1_Q1_output <- unite(PS1_Q1_output, date, c("Year", "Month"), sep="")
  PS1_Q1_output$date <- as.integer(PS1_Q1_output$date)
  PS1Q1_FF_merged <- merge(PS1_Q1_output, FF_mkt)
  PS1Q1_FF_merged <- separate(PS1Q1_FF_merged, date, into = c("Year", "Month"), sep=4)
  
  PS1Q1_FF_merged$Excess_Ret <- PS1Q1_FF_merged$Stock_Vw_Ret - PS1Q1_FF_merged$RF
  
  EstimatedExcessRet=PS1Q1_FF_merged[, prod(1+Excess_Ret)-1, by=Year]$V1
  ActualExcessRet=PS1Q1_FF_merged[, prod(1+`Mkt-RF`)-1, by=Year]$V1
  
  PS1_Q2_output <- matrix(nrow = 5, ncol = 2)
  rownames(PS1_Q2_output) <- c("Annualized Mean", "Annualized Standard Deviation", "Annualized Sharp Ratio", "Skewness", "Excess Kurtosis")
  colnames(PS1_Q2_output) <- c("Estimated FF Market Excess Return", "Actual FF Market Excess Return")
  
  PS1_Q2_output[, 1] <- c(mean(EstimatedExcessRet), sd(EstimatedExcessRet), mean(EstimatedExcessRet)/sd(EstimatedExcessRet), skewness(PS1Q1_FF_merged$Excess_Ret), kurtosis(PS1Q1_FF_merged$Excess_Ret)-3)
  PS1_Q2_output[, 2] <- c(mean(ActualExcessRet), sd(ActualExcessRet), mean(ActualExcessRet)/sd(ActualExcessRet), skewness(PS1Q1_FF_merged$`Mkt-RF`), kurtosis(PS1Q1_FF_merged$`Mkt-RF`)-3)
  PS1_Q2_output <- round(PS1_Q2_output, 4)
  
  return (PS1_Q2_output)
  
}

PS1_Q2_output <- PS1_Q2(CRSP_Stocks, FF_mkt)


### Task 3
Report (up to 8 decimal digits) the correlation between your time series and French’s time series,
and the maximum absolute difference between the two-time series. It is zero? If not, justify whether
the difference is economically negligible or not. What are the reasons a nonzero difference? You
should be comparing between July 1926 to December 2017, at a monthly frequency.

In [ ]:

PS1_Q3 <- function(Monthly_CRSP_Stocks, FF_mkt) {
  
  PS1_Q1_output <- PS1_Q1(CRSP_Stocks)
  PS1_Q1_output <- unite(PS1_Q1_output, date, c("Year", "Month"), sep="")
  PS1_Q1_output$date <- as.integer(PS1_Q1_output$date)
  PS1Q1_FF_merged <- merge(PS1_Q1_output, FF_mkt)
  PS1Q1_FF_merged <- separate(PS1Q1_FF_merged, date, into = c("Year", "Month"), sep=4)
  
  PS1Q1_FF_merged$Excess_Ret <- PS1Q1_FF_merged$Stock_Vw_Ret - PS1Q1_FF_merged$RF
  
  correlation_estimated_Actual <- cor(PS1Q1_FF_merged$`Mkt-RF`, PS1Q1_FF_merged$Excess_Ret) 
  MaxAbsDiff <- max(abs(PS1Q1_FF_merged$`Mkt-RF`- PS1Q1_FF_merged$Excess_Ret))
  
  PS1_Q3_output <- matrix(nrow = 2, ncol = 1)
  rownames(PS1_Q3_output) <- c("Correlation", "Maximum absolute difference")
  PS1_Q3_output[, 1 ] <- c(correlation_estimated_Actual, MaxAbsDiff)
  
  PS1_Q3_output <- round(PS1_Q3_output, 8)
  
  return(PS1_Q3_output)
}

PS1_Q3_output=PS1_Q3(Monthly_CRSP_Stocks, FF_mkt)

